# Exemplo 04: Aplicação Web Chainlit

Este notebook demonstra como criar e entender uma aplicação web moderna Chainlit para IA conversacional usando o Microsoft Foundry Local com o OpenAI SDK.

## Visão Geral

Chainlit é um framework poderoso para construir aplicações de IA conversacional com interfaces web modernas. Este exemplo mostra:

- 🌐 **Interface Web Moderna**: Interface de chat profissional com atualizações em tempo real
- 🔄 **Respostas em Streaming**: Transmissão de mensagens em tempo real para uma melhor experiência do usuário
- 🎯 **Integração com OpenAI SDK**: Configuração adequada do cliente API com Foundry Local
- 🛡️ **Tratamento de Erros**: Alternativas elegantes e mensagens de erro amigáveis ao usuário
- ⚙️ **Pronto para Produção**: Configuração e padrões de implantação de nível empresarial


## Pré-requisitos e Configuração

Antes de executar este notebook, certifique-se de que você possui os pacotes necessários:


In [14]:
# Install required packages
!pip install chainlit openai foundry-local-sdk

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


## Entendendo a Aplicação Chainlit

Vamos analisar a estrutura e os principais componentes da nossa aplicação Chainlit:


In [15]:
# Import necessary libraries
import chainlit as cl
import os
import sys
from typing import Optional, Dict, Any
from openai import OpenAI, AsyncOpenAI
import asyncio

# Check for Foundry Local SDK
try:
    from foundry_local import FoundryLocalManager
    FOUNDRY_SDK_AVAILABLE = True
    print("✅ Foundry Local SDK is available")
except ImportError:
    FOUNDRY_SDK_AVAILABLE = False
    print("⚠️ Foundry Local SDK not available, will use manual configuration")

print("📦 Libraries imported successfully")

✅ Foundry Local SDK is available
📦 Libraries imported successfully


## Classe de Configuração do Cliente

Esta classe gerencia a configuração do cliente OpenAI com integração Foundry Local:


In [16]:
class FoundryClientManager:
    """Manages OpenAI client setup for Foundry Local integration."""
    
    def __init__(self, model_name: str = "phi-4-mini"):
        self.model_name = model_name
        self.client = None
        self.async_client = None
        self.base_url = None
        self.api_key = None
        
    def _get_fallback_config(self) -> tuple[str, str]:
        """Get fallback configuration from environment variables."""
        base_url = os.getenv("BASE_URL", "http://localhost:8000")
        api_key = os.getenv("API_KEY", "")
        return base_url, api_key
    
    def initialize_clients(self) -> Dict[str, Any]:
        """Initialize both sync and async OpenAI clients."""
        if FOUNDRY_SDK_AVAILABLE:
            try:
                print(f"🔄 Initializing Foundry Local SDK with model: {self.model_name}...")
                manager = FoundryLocalManager(self.model_name)
                
                self.base_url = manager.endpoint
                self.api_key = manager.api_key
                
                # Create both sync and async clients
                self.client = OpenAI(
                    base_url=self.base_url,
                    api_key=self.api_key
                )
                
                self.async_client = AsyncOpenAI(
                    base_url=self.base_url,
                    api_key=self.api_key
                )
                
                print(f"✅ Foundry Local SDK initialized at {self.base_url}")
                return {
                    "status": "success",
                    "method": "foundry_sdk",
                    "base_url": self.base_url,
                    "model": self.model_name
                }
                
            except Exception as e:
                print(f"⚠️ Foundry SDK failed ({e}), falling back to manual configuration")
        
        # Fallback to manual configuration
        self.base_url, self.api_key = self._get_fallback_config()
        
        self.client = OpenAI(
            base_url=f"{self.base_url}/v1",
            api_key=self.api_key
        )
        
        self.async_client = AsyncOpenAI(
            base_url=f"{self.base_url}/v1",
            api_key=self.api_key
        )
        
        print(f"🔧 Manual configuration at {self.base_url}/v1")
        return {
            "status": "success",
            "method": "manual",
            "base_url": f"{self.base_url}/v1",
            "model": self.model_name
        }
    
    async def test_connection(self) -> Dict[str, Any]:
        """Test the connection to Foundry Local service."""
        try:
            # Try to list available models
            models = await self.async_client.models.list()
            available_models = [model.id for model in models.data]
            
            # Test with a simple completion
            response = await self.async_client.chat.completions.create(
                model=self.model_name,
                messages=[{"role": "user", "content": "Hello, are you working?"}],
                max_tokens=50
            )
            
            return {
                "status": "healthy",
                "available_models": available_models,
                "current_model": self.model_name,
                "test_response": response.choices[0].message.content,
                "base_url": self.base_url
            }
            
        except Exception as e:
            return {
                "status": "error",
                "error": str(e),
                "base_url": self.base_url
            }

# Create a client manager instance
client_manager = FoundryClientManager("phi-4-mini")
init_result = client_manager.initialize_clients()

print(f"\n📊 **Client Initialization Result:**")
for key, value in init_result.items():
    print(f"   {key.title()}: {value}")

🔄 Initializing Foundry Local SDK with model: phi-4-mini...
2025-09-23 21:43:21 - HTTP Request: GET http://127.0.0.1:51211/foundry/list "HTTP/1.1 200 OK"
2025-09-23 21:43:21 - HTTP Request: GET http://127.0.0.1:51211/openai/models "HTTP/1.1 200 OK"
2025-09-23 21:43:21 - HTTP Request: GET http://127.0.0.1:51211/openai/load/Phi-4-mini-instruct-cuda-gpu?ttl=600&ep= "HTTP/1.1 200 OK"
✅ Foundry Local SDK initialized at http://127.0.0.1:51211/v1

📊 **Client Initialization Result:**
   Status: success
   Method: foundry_sdk
   Base_Url: http://127.0.0.1:51211/v1
   Model: phi-4-mini


## Teste de Conexão

Vamos testar nossa conexão com o serviço Foundry Local:


In [17]:
# Test the connection asynchronously
async def test_service_connection():
    """Test connection to Foundry Local service."""
    print("🔍 **Testing Foundry Local Connection**")
    print("=" * 50)
    
    health_check = await client_manager.test_connection()
    
    if health_check["status"] == "healthy":
        print("✅ **Connection Status:** Healthy")
        print(f"🔗 **Base URL:** {health_check['base_url']}")
        print(f"🤖 **Current Model:** {health_check['current_model']}")
        print(f"💬 **Test Response:** {health_check['test_response']}")
        
        print(f"\n📋 **Available Models ({len(health_check['available_models'])}):**")
        for i, model in enumerate(health_check['available_models'], 1):
            current = " (current)" if model == health_check['current_model'] else ""
            print(f"   {i}. {model}{current}")
    else:
        print("❌ **Connection Status:** Error")
        print(f"🔗 **Base URL:** {health_check['base_url']}")
        print(f"⚠️ **Error:** {health_check['error']}")
        print("\n🔧 **Troubleshooting:**")
        print("1. Check if Foundry Local is running: foundry service status")
        print("2. Start a model: foundry model run phi-4-mini")
        print("3. Verify the endpoint URL and port")
    
    return health_check

# Run the connection test
connection_result = await test_service_connection()

🔍 **Testing Foundry Local Connection**
2025-09-23 21:43:24 - HTTP Request: GET http://127.0.0.1:51211/v1/models "HTTP/1.1 200 OK"
2025-09-23 21:43:24 - HTTP Request: POST http://127.0.0.1:51211/v1/chat/completions "HTTP/1.1 400 Bad Request"
2025-09-23 21:43:24 - HTTP Request: POST http://127.0.0.1:51211/v1/chat/completions "HTTP/1.1 400 Bad Request"
❌ **Connection Status:** Error
🔗 **Base URL:** http://127.0.0.1:51211/v1
⚠️ **Error:** Error code: 400

🔧 **Troubleshooting:**
1. Check if Foundry Local is running: foundry service status
2. Start a model: foundry model run phi-4-mini
3. Verify the endpoint URL and port


## Estrutura da Aplicação Chainlit

Agora vamos analisar os principais componentes da nossa aplicação Chainlit:


In [18]:
# Read the actual Chainlit application file
app_file_path = "../04/app.py"

try:
    with open(app_file_path, 'r', encoding='utf-8') as f:
        app_content = f.read()
    
    print("📄 **Chainlit Application Code:**")
    print("=" * 60)
    print(app_content)
    
except FileNotFoundError:
    print(f"❌ Application file not found at {app_file_path}")
    print("Let's create a sample application structure instead:")
    
    sample_app = '''
# Chainlit Application Structure

import chainlit as cl
from openai import AsyncOpenAI
from foundry_local import FoundryLocalManager

# Global client variable
client = None

@cl.on_chat_start
async def on_chat_start():
    """Initialize the chat session."""
    # Setup client and welcome user
    
@cl.on_message
async def on_message(message: cl.Message):
    """Handle incoming messages with streaming response."""
    # Process message and stream response
    
# Error handling and client setup functions...
'''
    print(sample_app)

📄 **Chainlit Application Code:**
#!/usr/bin/env python3
# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT License.

import os
import chainlit as cl
from openai import OpenAI

try:
    from foundry_local import FoundryLocalManager
    FOUNDRY_SDK_AVAILABLE = True
except ImportError:
    FOUNDRY_SDK_AVAILABLE = False

# Global variables for client and model
client = None
model_name = None


async def initialize_client():
    """Initialize OpenAI client with Foundry Local or fallback configuration."""
    global client, model_name
    
    alias = os.environ.get("MODEL", "phi-4-mini")
    
    if FOUNDRY_SDK_AVAILABLE:
        try:
            # Use FoundryLocalManager for proper service management
            manager = FoundryLocalManager(alias)
            model_info = manager.get_model_info(alias)
            
            # Configure OpenAI client to use local Foundry service
            client = OpenAI(
                base_url=manager.endpoint,
    

## Conceitos Principais do Chainlit

Vamos entender os principais conceitos utilizados nas aplicações Chainlit:


In [19]:
print("🎯 **Key Chainlit Concepts**")
print("=" * 50)

chainlit_concepts = {
    "@cl.on_chat_start": {
        "purpose": "Decorator for session initialization",
        "when_called": "When a new chat session begins",
        "typical_use": "Setup client, show welcome message, initialize context"
    },
    "@cl.on_message": {
        "purpose": "Decorator for message handling",
        "when_called": "When user sends a message",
        "typical_use": "Process user input, generate AI response, stream output"
    },
    "cl.Message": {
        "purpose": "Message object containing user input",
        "properties": "content, author, timestamp, elements",
        "typical_use": "Access user's message content and metadata"
    },
    "cl.make_async": {
        "purpose": "Convert sync functions to async",
        "when_needed": "When using sync OpenAI client in async context",
        "typical_use": "Wrap synchronous API calls for Chainlit compatibility"
    },
    "Streaming Response": {
        "purpose": "Real-time message updates",
        "implementation": "Create empty message, update content progressively",
        "typical_use": "Better UX for long responses, real-time feedback"
    }
}

for concept, details in chainlit_concepts.items():
    print(f"\n🔧 **{concept}**")
    for key, value in details.items():
        print(f"   {key.replace('_', ' ').title()}: {value}")

🎯 **Key Chainlit Concepts**

🔧 **@cl.on_chat_start**
   Purpose: Decorator for session initialization
   When Called: When a new chat session begins
   Typical Use: Setup client, show welcome message, initialize context

🔧 **@cl.on_message**
   Purpose: Decorator for message handling
   When Called: When user sends a message
   Typical Use: Process user input, generate AI response, stream output

🔧 **cl.Message**
   Purpose: Message object containing user input
   Properties: content, author, timestamp, elements
   Typical Use: Access user's message content and metadata

🔧 **cl.make_async**
   Purpose: Convert sync functions to async
   When Needed: When using sync OpenAI client in async context
   Typical Use: Wrap synchronous API calls for Chainlit compatibility

🔧 **Streaming Response**
   Purpose: Real-time message updates
   Implementation: Create empty message, update content progressively
   Typical Use: Better UX for long responses, real-time feedback


## Implementação de Respostas em Streaming

Veja como as respostas em streaming funcionam no Chainlit:


In [20]:
async def demonstrate_streaming_pattern():
    """Demonstrate the streaming response pattern used in Chainlit."""
    print("🌊 **Streaming Response Pattern**")
    print("=" * 50)
    
    # This is how streaming works in the actual Chainlit app
    streaming_code = '''
# 1. Create an empty message to update progressively
msg = cl.Message(content="")
await msg.send()

# 2. Make streaming API call
stream = await client.chat.completions.create(
    model=model_name,
    messages=messages,
    stream=True,
    max_tokens=1000
)

# 3. Process each chunk and update the message
async for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        await msg.stream_token(chunk.choices[0].delta.content)

# 4. Finalize the message
await msg.update()
'''
    
    print("📝 **Streaming Implementation:**")
    print(streaming_code)
    
    print("\n✨ **Benefits of Streaming:**")
    benefits = [
        "🚀 **Real-time feedback**: Users see responses as they're generated",
        "⚡ **Better perceived performance**: Feels faster than waiting for complete response",
        "🎯 **User engagement**: Keeps users engaged during long responses",
        "🛑 **Early termination**: Users can interrupt if response goes off-track",
        "💡 **Professional UX**: Modern chat interface experience"
    ]
    
    for benefit in benefits:
        print(f"   {benefit}")
    
    print("\n🔧 **Implementation Notes:**")
    notes = [
        "Always use AsyncOpenAI for Chainlit applications",
        "Handle streaming errors gracefully with try-catch blocks",
        "Check for None content in delta chunks",
        "Update message when streaming is complete",
        "Consider rate limiting for production deployments"
    ]
    
    for note in notes:
        print(f"   • {note}")

await demonstrate_streaming_pattern()

🌊 **Streaming Response Pattern**
📝 **Streaming Implementation:**

# 1. Create an empty message to update progressively
msg = cl.Message(content="")
await msg.send()

# 2. Make streaming API call
stream = await client.chat.completions.create(
    model=model_name,
    messages=messages,
    stream=True,
    max_tokens=1000
)

# 3. Process each chunk and update the message
async for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        await msg.stream_token(chunk.choices[0].delta.content)

# 4. Finalize the message
await msg.update()


✨ **Benefits of Streaming:**
   🚀 **Real-time feedback**: Users see responses as they're generated
   ⚡ **Better perceived performance**: Feels faster than waiting for complete response
   🎯 **User engagement**: Keeps users engaged during long responses
   🛑 **Early termination**: Users can interrupt if response goes off-track
   💡 **Professional UX**: Modern chat interface experience

🔧 **Implementation Notes:**
   • Always use Asyn

## Padrões de Tratamento de Erros

Um tratamento de erros robusto é essencial para aplicações Chainlit em produção:


In [21]:
async def demonstrate_error_handling():
    """Show error handling patterns for Chainlit applications."""
    print("🛡️ **Error Handling Patterns**")
    print("=" * 50)
    
    error_scenarios = {
        "Client Initialization Failure": {
            "cause": "Foundry Local service not running",
            "handling": "Graceful fallback to manual configuration",
            "user_message": "Service initializing, please wait..."
        },
        "Model Not Available": {
            "cause": "Requested model not loaded",
            "handling": "Try alternative models or suggest model loading",
            "user_message": "Model unavailable, trying alternative..."
        },
        "Network Connection Error": {
            "cause": "Network issues or service down",
            "handling": "Retry with exponential backoff",
            "user_message": "Connection issue, retrying..."
        },
        "Streaming Interruption": {
            "cause": "Stream ends unexpectedly",
            "handling": "Complete partial response gracefully",
            "user_message": "Response completed (partial)"
        },
        "Rate Limiting": {
            "cause": "Too many requests",
            "handling": "Queue requests or ask user to wait",
            "user_message": "High traffic, please wait a moment..."
        }
    }
    
    for scenario, details in error_scenarios.items():
        print(f"\n⚠️ **{scenario}**")
        print(f"   🔍 Cause: {details['cause']}")
        print(f"   🔧 Handling: {details['handling']}")
        print(f"   💬 User Message: '{details['user_message']}'")
    
    print("\n📋 **Error Handling Best Practices:**")
    best_practices = [
        "🎯 **User-Friendly Messages**: Never show technical errors to users",
        "🔄 **Automatic Retry**: Implement retry logic for transient failures",
        "📊 **Logging**: Log errors for debugging while keeping user experience smooth",
        "🛠️ **Graceful Degradation**: Provide limited functionality when services are down",
        "💡 **Helpful Suggestions**: Guide users on how to resolve issues",
        "⚡ **Fast Failure**: Fail quickly rather than letting users wait indefinitely"
    ]
    
    for practice in best_practices:
        print(f"   {practice}")
    
    # Show example error handling code
    print("\n💻 **Example Error Handling Code:**")
    error_code = '''
try:
    stream = await client.chat.completions.create(
        model=model_name,
        messages=messages,
        stream=True,
        max_tokens=1000
    )
    
    async for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            await msg.stream_token(chunk.choices[0].delta.content)
            
except Exception as e:
    error_msg = "I encountered an issue. Please try again."
    await cl.Message(content=error_msg, author="System").send()
    # Log the actual error for debugging
    print(f"Error: {e}")
'''
    print(error_code)

await demonstrate_error_handling()

🛡️ **Error Handling Patterns**

⚠️ **Client Initialization Failure**
   🔍 Cause: Foundry Local service not running
   🔧 Handling: Graceful fallback to manual configuration
   💬 User Message: 'Service initializing, please wait...'

⚠️ **Model Not Available**
   🔍 Cause: Requested model not loaded
   🔧 Handling: Try alternative models or suggest model loading
   💬 User Message: 'Model unavailable, trying alternative...'

⚠️ **Network Connection Error**
   🔍 Cause: Network issues or service down
   🔧 Handling: Retry with exponential backoff
   💬 User Message: 'Connection issue, retrying...'

⚠️ **Streaming Interruption**
   🔍 Cause: Stream ends unexpectedly
   🔧 Handling: Complete partial response gracefully
   💬 User Message: 'Response completed (partial)'

⚠️ **Rate Limiting**
   🔍 Cause: Too many requests
   🔧 Handling: Queue requests or ask user to wait
   💬 User Message: 'High traffic, please wait a moment...'

📋 **Error Handling Best Practices:**
   🎯 **User-Friendly Messages**: Nev

## Executando a Aplicação Chainlit

Veja como executar e implantar a aplicação Chainlit:


In [22]:
print("🚀 **Running Chainlit Application**")
print("=" * 50)

running_instructions = {
    "Development Mode": {
        "command": "chainlit run app.py --watch",
        "description": "Auto-reload on file changes",
        "port": "8000 (default)",
        "use_case": "Local development and testing"
    },
    "Production Mode": {
        "command": "chainlit run app.py --host 0.0.0.0 --port 8080",
        "description": "Production deployment",
        "port": "8080 (configurable)",
        "use_case": "Server deployment"
    },
    "Custom Configuration": {
        "command": "chainlit run app.py -h 0.0.0.0 -p 3000 --no-cache",
        "description": "Custom host, port, and caching options",
        "port": "3000 (custom)",
        "use_case": "Specific deployment requirements"
    }
}

for mode, config in running_instructions.items():
    print(f"\n🔧 **{mode}**")
    for key, value in config.items():
        print(f"   {key.title()}: {value}")

print("\n🌐 **Access Points:**")
access_info = [
    "📱 **Local Development**: http://localhost:8000",
    "🌍 **Network Access**: http://YOUR_IP:8000",
    "🔗 **Production**: https://your-domain.com",
    "📊 **Health Check**: Add /health endpoint for monitoring"
]

for info in access_info:
    print(f"   {info}")

print("\n⚙️ **Environment Variables for Production:**")
env_vars = {
    "BASE_URL": "http://localhost:8000 (Foundry Local endpoint)",
    "API_KEY": "your-api-key (if required)",
    "MODEL_NAME": "phi-4-mini (default model)",
    "MAX_TOKENS": "1000 (response length limit)",
    "CHAINLIT_HOST": "0.0.0.0 (production host)",
    "CHAINLIT_PORT": "8080 (production port)"
}

for var, desc in env_vars.items():
    print(f"   {var}={desc}")

🚀 **Running Chainlit Application**

🔧 **Development Mode**
   Command: chainlit run app.py --watch
   Description: Auto-reload on file changes
   Port: 8000 (default)
   Use_Case: Local development and testing

🔧 **Production Mode**
   Command: chainlit run app.py --host 0.0.0.0 --port 8080
   Description: Production deployment
   Port: 8080 (configurable)
   Use_Case: Server deployment

🔧 **Custom Configuration**
   Command: chainlit run app.py -h 0.0.0.0 -p 3000 --no-cache
   Description: Custom host, port, and caching options
   Port: 3000 (custom)
   Use_Case: Specific deployment requirements

🌐 **Access Points:**
   📱 **Local Development**: http://localhost:8000
   🌍 **Network Access**: http://YOUR_IP:8000
   🔗 **Production**: https://your-domain.com
   📊 **Health Check**: Add /health endpoint for monitoring

⚙️ **Environment Variables for Production:**
   BASE_URL=http://localhost:8000 (Foundry Local endpoint)
   API_KEY=your-api-key (if required)
   MODEL_NAME=phi-4-mini (defaul

## Opções de Personalização

O Chainlit oferece diversas opções de personalização:


In [23]:
print("🎨 **Chainlit Customization Options**")
print("=" * 50)

customization_areas = {
    "UI Customization": {
        "config_file": ".chainlit/config.toml",
        "options": [
            "Custom theme colors and fonts",
            "Company logo and branding",
            "Custom CSS styling",
            "Dark/light mode preferences"
        ]
    },
    "Chat Features": {
        "config_file": "app.py (programmatic)",
        "options": [
            "File upload support",
            "Image and media handling",
            "Custom message elements",
            "Action buttons and quick replies"
        ]
    },
    "Authentication": {
        "config_file": "auth.py + config.toml",
        "options": [
            "OAuth integration (Google, GitHub)",
            "LDAP/Active Directory",
            "Custom authentication providers",
            "Role-based access control"
        ]
    },
    "Deployment": {
        "config_file": "docker-compose.yml / Dockerfile",
        "options": [
            "Docker containerization",
            "Kubernetes deployment",
            "Cloud platform integration",
            "Reverse proxy configuration"
        ]
    }
}

for area, details in customization_areas.items():
    print(f"\n🔧 **{area}**")
    print(f"   📄 Config: {details['config_file']}")
    print(f"   ⚙️ Options:")
    for option in details['options']:
        print(f"      • {option}")

# Show sample configuration
print("\n📝 **Sample .chainlit/config.toml:**")
sample_config = '''
[project]
name = "Foundry Local Chat"
author = "Your Organization"
description = "AI Chat powered by Foundry Local"

[UI]
name = "Foundry AI Assistant"
show_readme_as_default = true
show_cloud_icon = false

[theme]
primary_color = "#0078d4"
background_color = "#ffffff"
text_color = "#323130"

[features]
allow_unsafe_html = false
max_message_size = 4096
max_file_size_mb = 10
'''
print(sample_config)

🎨 **Chainlit Customization Options**

🔧 **UI Customization**
   📄 Config: .chainlit/config.toml
   ⚙️ Options:
      • Custom theme colors and fonts
      • Company logo and branding
      • Custom CSS styling
      • Dark/light mode preferences

🔧 **Chat Features**
   📄 Config: app.py (programmatic)
   ⚙️ Options:
      • File upload support
      • Image and media handling
      • Custom message elements
      • Action buttons and quick replies

🔧 **Authentication**
   📄 Config: auth.py + config.toml
   ⚙️ Options:
      • OAuth integration (Google, GitHub)
      • LDAP/Active Directory
      • Custom authentication providers
      • Role-based access control

🔧 **Deployment**
   📄 Config: docker-compose.yml / Dockerfile
   ⚙️ Options:
      • Docker containerization
      • Kubernetes deployment
      • Cloud platform integration
      • Reverse proxy configuration

📝 **Sample .chainlit/config.toml:**

[project]
name = "Foundry Local Chat"
author = "Your Organization"
description = 

## Recursos Avançados

Explore os recursos avançados do Chainlit para aplicações em produção:


In [24]:
print("🚀 **Advanced Chainlit Features**")
print("=" * 50)

advanced_features = {
    "Session Management": {
        "description": "Maintain conversation context across messages",
        "implementation": "cl.user_session for storing state",
        "use_cases": ["Multi-turn conversations", "User preferences", "Context persistence"]
    },
    "File Uploads": {
        "description": "Handle document uploads and processing",
        "implementation": "@cl.on_file_upload decorator",
        "use_cases": ["Document analysis", "Image processing", "Data ingestion"]
    },
    "Action Buttons": {
        "description": "Interactive buttons for user actions",
        "implementation": "cl.Action elements",
        "use_cases": ["Quick replies", "Workflow triggers", "Menu navigation"]
    },
    "Data Persistence": {
        "description": "Store conversation history and user data",
        "implementation": "Database integration",
        "use_cases": ["Chat history", "User analytics", "Feedback collection"]
    },
    "Multi-modal Support": {
        "description": "Handle text, images, and other media",
        "implementation": "cl.Image, cl.File elements",
        "use_cases": ["Visual Q&A", "Document chat", "Media analysis"]
    }
}

for feature, details in advanced_features.items():
    print(f"\n🎯 **{feature}**")
    print(f"   📝 Description: {details['description']}")
    print(f"   🔧 Implementation: {details['implementation']}")
    print(f"   💡 Use Cases:")
    for use_case in details['use_cases']:
        print(f"      • {use_case}")

# Show example code for session management
print("\n💻 **Session Management Example:**")
session_code = '''
@cl.on_chat_start
async def on_chat_start():
    # Initialize session state
    cl.user_session.set("conversation_history", [])
    cl.user_session.set("user_preferences", {"temperature": 0.7})

@cl.on_message
async def on_message(message: cl.Message):
    # Get session state
    history = cl.user_session.get("conversation_history", [])
    preferences = cl.user_session.get("user_preferences", {})
    
    # Add current message to history
    history.append({"role": "user", "content": message.content})
    
    # Use full conversation context
    response = await client.chat.completions.create(
        model="phi-4-mini",
        messages=history,
        temperature=preferences.get("temperature", 0.7)
    )
    
    # Update session with AI response
    history.append({"role": "assistant", "content": response.choices[0].message.content})
    cl.user_session.set("conversation_history", history)
'''
print(session_code)

🚀 **Advanced Chainlit Features**

🎯 **Session Management**
   📝 Description: Maintain conversation context across messages
   🔧 Implementation: cl.user_session for storing state
   💡 Use Cases:
      • Multi-turn conversations
      • User preferences
      • Context persistence

🎯 **File Uploads**
   📝 Description: Handle document uploads and processing
   🔧 Implementation: @cl.on_file_upload decorator
   💡 Use Cases:
      • Document analysis
      • Image processing
      • Data ingestion

🎯 **Action Buttons**
   📝 Description: Interactive buttons for user actions
   🔧 Implementation: cl.Action elements
   💡 Use Cases:
      • Quick replies
      • Workflow triggers
      • Menu navigation

🎯 **Data Persistence**
   📝 Description: Store conversation history and user data
   🔧 Implementation: Database integration
   💡 Use Cases:
      • Chat history
      • User analytics
      • Feedback collection

🎯 **Multi-modal Support**
   📝 Description: Handle text, images, and other media
   

## Lista de Verificação para Implantação em Produção

Considerações essenciais para implantar aplicações Chainlit em produção:


In [25]:
print("✅ **Production Deployment Checklist**")
print("=" * 50)

deployment_checklist = {
    "🔒 Security": [
        "Enable authentication and authorization",
        "Use HTTPS with proper SSL certificates",
        "Implement rate limiting and request validation",
        "Sanitize user inputs and prevent injection attacks",
        "Set up proper CORS policies",
        "Use environment variables for sensitive configuration"
    ],
    "⚡ Performance": [
        "Configure connection pooling for database",
        "Implement caching for frequent responses",
        "Set up load balancing for multiple instances",
        "Monitor memory usage and optimize where needed",
        "Configure appropriate timeout values",
        "Use CDN for static assets"
    ],
    "📊 Monitoring": [
        "Set up application logging and monitoring",
        "Configure health checks and uptime monitoring",
        "Track user engagement and conversation metrics",
        "Monitor API response times and error rates",
        "Set up alerting for critical issues",
        "Implement user feedback collection"
    ],
    "🛠️ Maintenance": [
        "Regular backups of conversation data",
        "Automated deployment pipelines",
        "Version control for configuration changes",
        "Documentation for troubleshooting",
        "Capacity planning and scaling procedures",
        "Update procedures for dependencies"
    ],
    "🌐 Infrastructure": [
        "Container orchestration (Docker/Kubernetes)",
        "Reverse proxy configuration (nginx/Apache)",
        "Database setup and optimization",
        "Network security and firewall rules",
        "Backup and disaster recovery plans",
        "Multi-region deployment for redundancy"
    ]
}

for category, items in deployment_checklist.items():
    print(f"\n{category}")
    for item in items:
        print(f"   ☐ {item}")

print("\n🚀 **Quick Production Setup Commands:**")
commands = [
    "# Build Docker image",
    "docker build -t chainlit-app .",
    "",
    "# Run with production settings",
    "docker run -d -p 8080:8080 -e NODE_ENV=production chainlit-app",
    "",
    "# Health check",
    "curl http://localhost:8080/health",
    "",
    "# Monitor logs",
    "docker logs -f chainlit-app"
]

for cmd in commands:
    if cmd.startswith("#"):
        print(f"\n💡 {cmd}")
    elif cmd:
        print(f"   {cmd}")
    else:
        print()

✅ **Production Deployment Checklist**

🔒 Security
   ☐ Enable authentication and authorization
   ☐ Use HTTPS with proper SSL certificates
   ☐ Implement rate limiting and request validation
   ☐ Sanitize user inputs and prevent injection attacks
   ☐ Set up proper CORS policies
   ☐ Use environment variables for sensitive configuration

⚡ Performance
   ☐ Configure connection pooling for database
   ☐ Implement caching for frequent responses
   ☐ Set up load balancing for multiple instances
   ☐ Monitor memory usage and optimize where needed
   ☐ Configure appropriate timeout values
   ☐ Use CDN for static assets

📊 Monitoring
   ☐ Set up application logging and monitoring
   ☐ Configure health checks and uptime monitoring
   ☐ Track user engagement and conversation metrics
   ☐ Monitor API response times and error rates
   ☐ Set up alerting for critical issues
   ☐ Implement user feedback collection

🛠️ Maintenance
   ☐ Regular backups of conversation data
   ☐ Automated deployment p

## Resumo e Melhores Práticas

Este notebook abordou todo o processo de desenvolvimento de uma aplicação Chainlit:

### ✅ Componentes Principais Abordados

1. **🔧 Configuração do Cliente**: Integração com o SDK Local do Foundry e configuração de fallback
2. **🌊 Respostas em Streaming**: Atualizações de mensagens em tempo real para uma melhor experiência do usuário
3. **🛡️ Tratamento de Erros**: Gerenciamento de falhas de forma elegante e mensagens amigáveis ao usuário
4. **⚙️ Configuração**: Configuração baseada em ambiente e opções de personalização
5. **🚀 Implantação**: Padrões de implantação prontos para produção e melhores práticas

### 🎯 Arquitetura da Aplicação Chainlit

```
User Browser ←→ Chainlit UI ←→ Python Backend ←→ Foundry Local ←→ AI Model
      ↓              ↓              ↓              ↓            ↓
   Web UI      Event Handlers   OpenAI Client   HTTP API    Local GPU
```

### 💡 Resumo de Melhores Práticas

- **🔄 Sempre Use Async**: Chainlit exige funções assíncronas para operações não bloqueantes
- **🌊 Implemente Streaming**: Oferece uma experiência de usuário melhor do que esperar por respostas completas
- **🛡️ Trate Erros de Forma Elegante**: Nunca exiba erros técnicos para os usuários finais
- **📊 Monitore o Desempenho**: Acompanhe tempos de resposta e métricas de engajamento dos usuários
- **🔒 Segurança como Padrão**: Implemente autenticação e validação de entrada desde o início
- **⚡ Otimize para Escalabilidade**: Projete para múltiplos usuários simultâneos desde o primeiro dia

### 🚀 Próximos Passos

- **📱 Suporte Multi-Modal**: Adicione capacidades de processamento de imagens e documentos
- **🤖 Integração com Agentes**: Conecte-se a sistemas multi-agentes para fluxos de trabalho complexos
- **📊 Painel de Analytics**: Construa interfaces administrativas para monitoramento e gerenciamento
- **🔧 Plugins Personalizados**: Desenvolva elementos e integrações personalizados para Chainlit
- **🌐 Integração com API**: Conecte-se a serviços externos e bancos de dados

Esta aplicação Chainlit demonstra como construir interfaces de IA conversacional prontas para produção, aproveitando o poder de modelos de IA locais através do Microsoft Foundry Local, enquanto oferece uma experiência de usuário moderna e responsiva.
